# Delineate basins
This script is the final version of a work-in-progress script that evolved over time. The work-in-progress script had a dedicated section that would allow manual re-runs of individual basins as part of basin delineation quality control. This QC approach was instrumental in creating the `2_manually_define_outlets.ipynb` script that precedes this notebook, and the "delineation confidence notes" that are part of this notebook. 

This work-in-progress script can be seen in older versions of the repo as `99_delineate_basins_one_by_one.ipynb`.

In [1]:
import sys
import shutil
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path     = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path  = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name  = cs.read_from_config(config_file,'cs_meta_name')

# MERIT Hydro data
merit_path = cs.read_from_config(config_file, 'merit_path')
merit_acc = Path(data_path) / merit_path / 'flow_accumulation_grid' / 'merit_hydro_acc.tif'
merit_dir = Path(data_path) / merit_path / 'flow_direction_grid' / 'merit_hydro_dir.tif'
merit_bas = Path(data_path) / merit_path / 'shapes' / 'basins' / 'cat_pfaf_7_8_MERIT_Hydro_v07_Basins_v01_bugfix1_hillslopes_pfaf_7_8_clean_fixed.shp'
merit_riv = Path(data_path) / merit_path / 'shapes' / 'rivers' / 'riv_pfaf_7_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp'

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')

# CRS to use for area calculations and overlap
crs = cs.read_from_config(config_file, 'equal_area_crs')

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Shapefiles
basins_shp = gpd.read_file(merit_bas)
rivers_shp = gpd.read_file(merit_riv)

In [6]:
# Fix up the missing hillslopes
# We are missing out on a single coastal hillslope in northern canada. Chances are we can complete the procedure anyway
arctic_hill_files = [Path(data_path) / merit_path / 'shapes' / 'basins' / 'hillslope_84_clean_fixed.shp',
                     Path(data_path) / merit_path / 'shapes' / 'basins' / 'hillslope_85_clean_fixed.shp']

arctic_hill_84 = gpd.read_file(arctic_hill_files[0])
arctic_hill_85 = gpd.read_file(arctic_hill_files[1])

basins_shp = pd.concat([basins_shp,arctic_hill_84,arctic_hill_85])

### Delineation

In [7]:
# Create a temporary directory for subsetting files
tmp_dir = Path(data_path) / 'tmp'
tmp_dir.mkdir(parents=True, exist_ok=True)

In [8]:
# Make sure we know the correct metadata columns
c_area = np.where(cs_meta.columns == 'Basin_area_km2')[0][0]
c_over = np.where(cs_meta.columns == 'Ref_and_new_shape_fractional_overlap')[0][0]

assert (cs_meta.columns[c_area] == 'Basin_area_km2') # If these aren't true, they'll pop an error
assert (cs_meta.columns[c_over] == 'Ref_and_new_shape_fractional_overlap')

In [9]:
def delineation_loop(i,plot=False):
    
    '''Delineates basin for station at index i in the CAMELS-spat metadata file'''
    
    # Input:
    # i - index in CAMELS-spat metadata file to process
    
    # 1. Prepare file paths and names
    # --------------------------------------------------------------------------------------
    basin_id, shp_lump_path, shp_dist_path, ref_path, plot_path = \
                                        cs.prepare_delineation_outputs(cs_meta, i, Path(data_path)/cs_basin_folder)
    print('Now running basin {}. {}'.format(i,basin_id))
    
    # 2. Get the station or outlet location
    # --------------------------------------------------------------------------------------
    lat,lon = cs.read_delineation_coords(cs_meta,i)
    #print('!!! Running with fixed lat/lon coordinates !!!'); lon,lat = -65.15583,47.70504
    
    # 3. Delineate the basin
    # --------------------------------------------------------------------------------------
    # 3a. Subset the Geotiffs for quicker processing
    subset_window = cs.determine_pysheds_data_loading_window(lat,lon, str(merit_acc))
    temp_acc,temp_dir = cs.subset_tifs_around_gauge(subset_window, 
                                                    str(merit_acc), 
                                                    str(merit_dir),
                                                    temp_dir = tmp_dir)
    grid,acc,fdir = cs.load_tifs_with_pysheds(str(temp_acc),str(temp_dir)) 
        
    # 3b. Basin extraction
    # Ignore future warnings for this part of the code to avoid FutureWarning screen spam by Shapely and Pandas
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        # Extract the full basin with pysheds
        lump_basin = cs.delineate_catchment_with_pysheds(grid,lon,lat,fdir,shp_lump_path) # Saves file as part of function
        lump_basin = cs.add_area_to_shape(lump_basin, column='unitarea', crs=crs)
        
        # Extract the MERIT Hydro basins and river network
        dist_basin, dist_river = cs.subset_merit_hydro_to_basin(basins_shp,rivers_shp,lump_basin,shp_dist_path,
                                                                lat,lon,crs=crs) # Ditto
    
    # 4. Delineation checks
    # --------------------------------------------------------------------------------------
    # 4a. Assert shape validity
    if not all(lump_basin.is_valid): print('ERROR: Lumped shape not valid') # assert() throws SyntaxWarning over .is_valid
    if not all(dist_basin.is_valid): print('ERROR: Distributed shape not valid')
        
    # 4b. Get areas and overlap
    area_lump = (lump_basin.to_crs('ESRI:102008').area / 10**6)[0] # [km^2]
    area_dist = dist_basin['unitarea'].sum() # [km^2]
    overlap_lump = cs.calculate_basin_and_reference_overlap(lump_basin, ref_path, crs=crs) # Returns 'n/a' if no reference shape
    overlap_dist = cs.calculate_basin_and_reference_overlap(dist_basin, ref_path, crs=crs) # Should be identical to lumped
    
    # 4c. Visual assessment of outcomes
    ref_areas = cs.get_reference_areas(cs_meta,i) # [km^2] 
    stats = cs.prepare_plotting_stats(ref_areas,area_lump,area_dist,overlap_lump,overlap_dist) 
    #if plot:
    #    cs.plot_discretization_results(basin_id, lump_basin, dist_basin, dist_river, ref_path, lat, lon, stats, plot_path)
    cs.plot_discretization_results(basin_id, lump_basin, dist_basin, dist_river, ref_path, lat, lon, stats, \
                                   plot_path, to_screen=plot)
    
    # 4d. Update metadata file
    cs_meta.iat[i,c_area] = area_dist
    cs_meta.iat[i,c_over] = overlap_dist
    
    # 5 Clean up - no guarantee this will do anything, but who knows
    del temp_acc, temp_dir, grid, acc, fdir, lump_basin, dist_basin, dist_river
    
    return

In [10]:
for i,_ in cs_meta.iterrows():
    delineation_loop(i)

Now running basin 0. CAN_01AD002
Now running basin 1. CAN_01AD003
Now running basin 2. CAN_01AE001
Now running basin 3. CAN_01AF007
Now running basin 4. CAN_01AF009
Now running basin 5. CAN_01AJ003
Now running basin 6. CAN_01AJ004
Now running basin 7. CAN_01AJ010
Now running basin 8. CAN_01AK001
Now running basin 9. CAN_01AK006


C:\Globus endpoint\CAMELS_spat\camels-spat-env\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Now running basin 10. CAN_01AK007
Now running basin 11. CAN_01AL002
Now running basin 12. CAN_01AL004


C:\Globus endpoint\CAMELS_spat\camels-spat-env\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Now running basin 13. CAN_01AM001
Now running basin 14. CAN_01AN002
Now running basin 15. CAN_01AP002
Now running basin 16. CAN_01AP004
Now running basin 17. CAN_01AP006
Now running basin 18. CAN_01AQ001
Now running basin 19. CAN_01BC001
Now running basin 20. CAN_01BD008
Now running basin 21. CAN_01BE001
Now running basin 22. CAN_01BG005
Now running basin 23. CAN_01BG008
Now running basin 24. CAN_01BG009
Now running basin 25. CAN_01BH005
Now running basin 26. CAN_01BH010
Now running basin 27. CAN_01BJ003
Now running basin 28. CAN_01BJ007
Now running basin 29. CAN_01BJ010
Now running basin 30. CAN_01BJ012
Now running basin 31. CAN_01BL002
Now running basin 32. CAN_01BL003
Now running basin 33. CAN_01BO001
Now running basin 34. CAN_01BP001
Now running basin 35. CAN_01BP002
Now running basin 36. CAN_01BQ001
Now running basin 37. CAN_01BS001
Now running basin 38. CAN_01BU002
Now running basin 39. CAN_01BU009


...

Now running basin 1674. USA_14139800
Now running basin 1675. USA_14141500
Now running basin 1676. USA_14154500
Now running basin 1677. USA_14158500
Now running basin 1678. USA_14158790
Now running basin 1679. USA_14166500
Now running basin 1680. USA_14182500
Now running basin 1681. USA_14185000
Now running basin 1682. USA_14185900
Now running basin 1683. USA_14187000
Now running basin 1684. USA_14216500
Now running basin 1685. USA_14222500
Now running basin 1686. USA_14236200
Now running basin 1687. USA_14301000
Now running basin 1688. USA_14303200


C:\Globus endpoint\CAMELS_spat\camels-spat-env\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Now running basin 1689. USA_14305500
Now running basin 1690. USA_14306340


C:\Globus endpoint\CAMELS_spat\camels-spat-env\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Now running basin 1691. USA_14306500
Now running basin 1692. USA_14308990
Now running basin 1693. USA_14309500
Now running basin 1694. USA_14316700
Now running basin 1695. USA_14325000
Now running basin 1696. USA_14362250
Now running basin 1697. USA_14400000


In [11]:
# Remove the temporary directory
shutil.rmtree(tmp_dir)

### Update metadata with confidence ratings

In [12]:
# Add a Delineation_confidence column
# Set the default to high, and update below in cases where we don't like the delineation that MERIT data gives us, but we
#   cannot easily fix this
cs_meta['Delineation_confidence'] = 'high'
cs_meta['Delineation_notes'] = 'n/a'

In [13]:
# Define the notes we want to make
# The will go in a list that we will later integrate into the metadata dataframe
# [country, gauge, confidence, note]
# Confidence:
# - low:    I believe the MERIT DEM is somehow wrong. I'll also asign 'low' if it's not clear if MERIT is wrong, 
#             but difference with reference shape feels too large
# - medium: uncertain to say which DEM (MERIT or whatever was used for the reference shape) is correct
# - high:   I believe the MERIT delineation is OK
# This is based on visual inspection of the MERIT accumulated area GeoTIFF, reference shape, and Google Maps satellite imagery
#  using a threshold of 90% area overlap with reference shape to initiate a manual check
notes = []
notes.append(['CAN','01AK006', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','01AL004', 'low',    'Very flat region, DEM accuracy is debatable due to potholes'])
notes.append(['CAN','01ED007', 'low',    'Very flat region, DEM accuracy is debatable due to potholes'])
notes.append(['CAN','01EE005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02AB014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02AB017', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02AC001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02AE001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02BA003', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of marshland; see 02BB003'])
notes.append(['CAN','02BB003', 'medium', 'Very flat region, MERIT DEM connects part of this basin to another outlet through a region of marshland; see 02BA003'])
notes.append(['CAN','02BF005', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF005', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF007', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF008', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF009', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF012', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02BF013', 'high',   'Part of a series of nested gauges: 02BF005, 02BF006, 02BF013, 02BF007, 02BF008, 02BF009, 02BF012'])
notes.append(['CAN','02CA002', 'low',    'Very flat region, MERIT DEM connects out parts of this basin to other basins by mistakenly identifying roads as rivers'])
notes.append(['CAN','02CF011', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','02CG003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02DD013', 'medium', 'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','02DD015', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','02ED013', 'medium', 'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','02FA002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02FC017', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GC010', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GC021', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GC031', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GD004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GD019', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GH002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02GH003', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['CAN','02HB012', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02HB022', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02HB023', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','02LB008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02LB017', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02PA007', 'medium', 'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','02PC018', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','02PD002', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02PD004', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02PD012', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02PD013', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02PD014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02PD015', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences. Part of a series of nested gauges: 02PD002, 02PD004, 02PD012, 02PD013, 02PD014, 02PD015'])
notes.append(['CAN','02UA003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02YA002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02YK005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02YK008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02YM004', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZD002', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road and due to presence of lakes'])
notes.append(['CAN','02ZC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZE004', 'medium', 'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road and due to presence of lakes'])
notes.append(['CAN','02ZG001', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','02ZG004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZH002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZL005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','02ZN002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','03BD002', 'unknown','No reference shape available'])
notes.append(['CAN','03ED001', 'unknown','No reference shape available'])
notes.append(['CAN','04CB001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','04EA001', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','04GB004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','04JC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','04JD005', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','04KA001', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','04LJ001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','04MD004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05AA008', 'low',    'Difference with reference shape likely explained by DEM not recognizing channel trajectory through a tunnel'])
notes.append(['CAN','05AC030', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05AH047', 'unknown','No reference shape available'])
notes.append(['CAN','05CB004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05CG004', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','05DA010', 'medium', 'Mountainous region, difference with reference shape likely explained by DEM differences due to presence of trees'])
notes.append(['CAN','05DF007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05EC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05ED002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05EE005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05EE006', 'low',    'Very flat region, MERIT DEM divides the reference shape into multiple subbasins with oulets in different directions'])
notes.append(['CAN','05EE009', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05EF005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05FA014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05FC007', 'low',    'Difference with reference shape likely explained by DEM not recognizing channel trajectory through a region of flat terrain'])
notes.append(['CAN','05FE002', 'medium', 'Very flat region, reference shape includes a subbasin that is identified as an endorheic basin in MERIT'])
notes.append(['CAN','05GA008', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','05GA010', 'medium', 'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road and due to presence of lakes'])
notes.append(['CAN','05GA012', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','05GB004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05GC007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05GD002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05HG021', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05HH003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JB004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JB007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JC004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JC007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JF011', 'medium', 'Very flat region, MERIT DEM divides the reference shape into multiple subbasins with oulets in different directions'])
notes.append(['CAN','05JF014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05JG001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05JG013', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05JH005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05JJ009', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05JK008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05JM010', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes; also likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05KB003', 'medium', 'Very flat region, reference shape includes subbasins that are identified as endorheic basins in MERIT. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05KB006', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05KB011', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05KC001', 'medium', 'Very flat region, reference shape includes subbasins that are identified as endorheic basins in MERIT. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05KE005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05KE007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05KH007', 'medium', 'Very flat region, reference shape includes subbasins that are identified as endorheic basins in MERIT. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05LA003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05LB004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05LB006', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05LB008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05LB010', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05LD001', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05LE011', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05LJ027', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05LL014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05LL015', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05LL027', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05MA020', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MA021', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MA025', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land. Part of a series of nested gauges: 05KB003, 05KB006, 05KB011, 05KC001, 05KH007, 05MA025'])
notes.append(['CAN','05MC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MC003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MD010', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05ME005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05ME007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MG001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MG003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05MH006', 'medium', 'Very flat region, MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','05MH007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05MH008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05NB033', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NB035', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05ND011', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NE003', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05NF002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NF006', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NF008', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05NF010', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road'])
notes.append(['CAN','05NG010', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NG012', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05NG020', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05OB016', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05OB021', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes and  built-up areas'])
notes.append(['CAN','05OC016', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05OC019', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05OC022', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05OE007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to the presence of roads'])
notes.append(['CAN','05OE009', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences due to the presence of roads'])
notes.append(['CAN','05OE010', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to the presence of roads'])
notes.append(['CAN','05OF009', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes and roads'])
notes.append(['CAN','05OF014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05OF017', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05OF018', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a road as a channel'])
notes.append(['CAN','05OF020', 'low',    'Very flat region, difference with reference shape due to DEM (mistakenly?) connecting rivers across a road instead of identifying the correct channel'])
notes.append(['CAN','05OH007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences partly due to the presence of roads'])
notes.append(['CAN','05OJ008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','05OJ016', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to the presence of roads'])
notes.append(['CAN','05OJ017', 'medium', 'Very flat region, difference with reference shape likely partly explained by DEM differences due to presence of  built-up areas'])
notes.append(['CAN','05QA004', 'low',    'Very flat region, difference with reference shape likely partly explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05RA001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05RB003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05RE002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05SC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05SD003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','05UA003', 'medium', 'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05UF004', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','05UH001', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','06AD008', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07AA007', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07AF010', 'high',   'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07AG003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07AH001', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07AH002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07BA002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07BB014', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences. MERIT river network does not pass through station location'])
notes.append(['CAN','07BC006', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07BE003', 'low',    'Very flat region, MERIT DEM does not have a stream near this station and also connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','07BE004', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences. Very large difference with reference shape'])
notes.append(['CAN','07BF002', 'low',    'Very flat region, difference with reference shape due to DEM mistakenly connecting rivers across a road instead of identifying the correct channel. Note that this leads to only relatively small differences with reference shape'])
notes.append(['CAN','07BJ001', 'low',    'Very flat region, difference with reference shape due to DEM mistakenly connecting rivers across a road instead of identifying the correct channel. Note that this leads to only relatively small differences with reference shape'])
notes.append(['CAN','07BK005', 'low',    'Very flat region, difference with reference shape due to DEM mistakenly connecting rivers across a road instead of identifying the correct channel. Note that this leads to only relatively small differences with reference shape'])
notes.append(['CAN','07CA003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07CA005', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07CA006', 'low',    'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying channels in 2 subbasins as land'])
notes.append(['CAN','07CA008', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07CD004', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07CE003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07DA018', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07DB001', 'medium', 'Very flat region, difference with reference shape likely (?) explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07EC003', 'unknown','No reference shape available'])
notes.append(['CAN','07FD013', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07GE003', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','07GE007', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07GH004', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['CAN','07HB001', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07HC002', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07JA003', 'medium', 'Very flat region, difference with reference shape partly due to DEM misidentifying the correct channel as land and also presence of lakes/potholes'])
notes.append(['CAN','07JC001', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07JC002', 'low',    'Very flat region, difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','07LB002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07OB006', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes. Large difference with reference shape'])
notes.append(['CAN','07SA002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','07SC002', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due to presence of lakes/potholes'])
notes.append(['CAN','08EE012', 'low',    'Moved gauge to correct DEM adding another subbasin to this outlet due to misidentifying the correct channel as land. Note that this leads to only relatively small differences with reference shape'])
notes.append(['CAN','08EE025', 'medium', 'Difference with reference shape likely explained by DEM differences due to presence of trees'])
notes.append(['CAN','08HA016', 'medium', 'Difference with reference shape likely explained by DEM differences due to presence of built-up areas'])
notes.append(['CAN','08HA068', 'medium', 'Difference with reference shape likely explained by DEM differences due to presence of trees'])
notes.append(['CAN','08HB048', 'medium', 'Difference with reference shape likely explained by DEM differences due to presence of trees'])
notes.append(['CAN','08HB075', 'low',    'Difference with reference shape likely explained by DEM differences due to presence of trees. Large difference with reference shape'])
notes.append(['CAN','08HF004', 'low',    'Difference with reference shape likely explained by DEM connecting another subbasin to this outlet due to misidentifying a channel as land. See 08HF005'])
notes.append(['CAN','08HF005', 'low',    'Difference with reference shape likely explained by DEM disconnecting another subbasin to this outlet due to misidentifying a channel as land. See 08HF004'])
notes.append(['CAN','08JD006', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','08JE004', 'low',    'Difference with reference shape likely explained by DEM connecting another subbasin to this outlet through a region with trees'])
notes.append(['CAN','08LE108', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08LG056', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08MH029', 'medium', 'Flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08NE008', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','08NE077', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08NG076', 'low',    'Difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['CAN','08NJ130', 'medium', 'Difference with reference shape likely explained by DEM differences due to presence of trees'])
notes.append(['CAN','08NM134', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08NM173', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08NM241', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08NM242', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08OA003', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','08OB002', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','09AC007', 'medium', 'MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','10AA004', 'medium', 'MERIT DEM connects another subbasin to this outlet through a region of flat terrain'])
notes.append(['CAN','10ED007', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','10ED009', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','10LC003', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['CAN','10QC001', 'unknown','No reference shape available'])
notes.append(['CAN','10RA001', 'unknown','No reference shape available'])
notes.append(['CAN','10TF001', 'medium', 'Flat regio, difference with reference shape likely explained by DEM differences'])
notes.append(['CAN','11AE008', 'low',    'Difference with reference shape explained by DEM misidentifying a channel as land'])
notes.append(['CAN','11AE014', 'medium', 'Difference with reference shape likely explained by DEM differences'])

notes.append(['USA','01118300', 'high',   'Gauge location is inside the reference shape. USGS (pers. comm., 2023-01-24) confims gauge location is accurate and reference shape is thus too large'])
notes.append(['USA','01121000', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','01123000', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','01195100', 'medium', 'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300). Also DEM differences.'])
notes.append(['USA','01466500', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','01484100', 'medium', 'Difference with reference shape likely explained by DEM differences due to the presence of a town'])
notes.append(['USA','01485500', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','01486000', 'low',    'Difference with reference shape likely explained by DEM differences due to various errors (channels misidentified, presumably flat landscape)'])
notes.append(['USA','01487000', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','01606500', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['USA','01620500', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','01658500', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','02011460', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','02038850', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','02055100', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','02202600', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02231000', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02231342', 'low',    'Very flat region, difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['USA','02235200', 'medium', 'Very flat region, had to move gauge substantially to get approximate match with reference shape'])
notes.append(['USA','02296500', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02297155', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02297310', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02298123', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02299950', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02300700', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences due the presence of a town'])
notes.append(['USA','02310947', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02310947', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02314500', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02315500', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02324400', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02327100', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02369800', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','02422500', 'medium', 'Difference with reference shape likely explained by DEM differences due the presence of a town'])
notes.append(['USA','02430615', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','03165000', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','03368000', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04115265', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04122200', 'low',    'Very flat region, difference with reference shape likely partly explained by DEM misidentifying a channel as land'])
notes.append(['USA','04122500', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04127918', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04197170', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04213000', 'medium', 'Very flat region, difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04213075', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04233000', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','04256000', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','05056000', 'medium', 'Difference with reference shape likely explained by DEM differences. Partly due to DEM containing non-contributing area'])
notes.append(['USA','05057000', 'medium', 'Difference with reference shape likely explained by DEM differences. Partly due to DEM containing non-contributing area and presence of lakes/potholes'])
notes.append(['USA','05087500', 'high',   'Difference with reference shape likely explained by DEM differences. Gauge location is also inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','05120500', 'medium', 'Difference with reference shape likely explained by DEM differences due to the presence of lakes/potholes'])
notes.append(['USA','05123400', 'low',    'Difference with reference shape likely explained by DEM differences due to the presence of lakes/potholes and DEM misidentifying a channel as land'])
notes.append(['USA','05525500', 'low',    'Difference with reference shape likely explained by DEM connecting rivers across a road'])
notes.append(['USA','05591550', 'low',    'Difference with reference shape likely explained by DEM misidentifying a channel as land'])
notes.append(['USA','05592050', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','05592575', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','06154410', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','06350000', 'high',   'Nested gauge of 6354000'])
notes.append(['USA','06353000', 'high',   'Nested gauge of 6354000'])
notes.append(['USA','06352000', 'high',   'Nested gauge of 6353000, 6354000'])
notes.append(['USA','06404000', 'high',   'Nested gauge of 6406000'])
notes.append(['USA','06408700', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','06440200', 'high',   'Nested gauge of 6441500'])
notes.append(['USA','06447000', 'high',   'Nested gauge of 6452000'])
notes.append(['USA','06447500', 'medium', 'Difference with reference shape likely explained by DEM differences. Nested gauge of 6450500, 6452000'])
notes.append(['USA','06450500', 'high',   'Nested gauge of 6452000'])
notes.append(['USA','06468170', 'high',   'Nested gauge of 6468250'])
notes.append(['USA','06479215', 'high',   'Nested gauge of 6479438'])
notes.append(['USA','06479438', 'low',    'Difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['USA','06614800', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','06746095', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','06784000', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','06889200', 'high',   'Nested gauge of 6889500'])
notes.append(['USA','07066000', 'high',   'Nested gauge of 7067000, 7068000'])
notes.append(['USA','07067000', 'high',   'Nested gauge of 7068000'])
notes.append(['USA','07142300', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','07196900', 'high',   'Nested gauge of 7197000'])
notes.append(['USA','07301410', 'high',   'Nested gauge of 7301500'])
notes.append(['USA','08029500', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','08066300', 'low',    'Difference with reference shape likely explained by DEM differences. Large difference with reference shape'])
notes.append(['USA','08070000', 'high',   'Nested gauge of 8070200'])
notes.append(['USA','08079600', 'medium', 'Difference with reference shape likely explained by DEM differences. Note the near-exact match of MERIT Hydro delineation with Gages II area'])
notes.append(['USA','08164600', 'medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','08377900', 'high',   'Nested gauge of 8378500'])
notes.append(['USA','09034900', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','09378630', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','09480000', 'high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','09492400', 'high',   'Nested gauge of 9494000'])
notes.append(['USA','10023000','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10166430','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10172700','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10172800','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10205030','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10242000','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10249300','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10258000','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10258500','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','10259200','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10263500','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10310500','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','10316500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10329500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10336660','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','10336740','medium', 'Difference with reference shape likely explained by DEM differences. Note the near-exact match of MERIT Hydro delineation with Gages II area'])
notes.append(['USA','10348850','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','10396000','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','11098000','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','11180500','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','11180960','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','11224500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','11253310','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','11264500','high',   'Nested gauge of 11266500'])
notes.append(['USA','11274500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','11274630','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','11299600','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','11383500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','12095000','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','12114500','high',   'Nested gauge of 12115000'])
notes.append(['USA','12115500','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','12117000','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','12143600','high',   'Nested gauge of 12144000'])
notes.append(['USA','12147600','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','12186000','high',   'Nested gauge of 12189500'])
notes.append(['USA','12388400','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','13083000','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','13337000','high',   'Nested gauge of 13340000'])
notes.append(['USA','13338500','high',   'Nested gauge of 13340000'])
notes.append(['USA','14092750','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','14138870','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','14158500','high',   'Gauge location is inside the reference shape. This reference shape is likely incorrect'])
notes.append(['USA','14158790','medium', 'Difference with reference shape likely explained by DEM differences'])
notes.append(['USA','14303200','high',   'Gauge location is inside the reference shape. Assuming the reference shape is too large here (see: USA 1118300)'])
notes.append(['USA','14306340','high',   'Nested gauge of 14306500'])

In [14]:
# Ensure we have the correct column IDs
c_note, c_conf = 25,24
assert (cs_meta.columns[c_note] == 'Delineation_notes') # If these aren't true, they'll pop an error
assert (cs_meta.columns[c_conf] == 'Delineation_confidence')

In [15]:
# Apply the changes to the metadata file
for note in notes:
    
    country = note[0]
    station = note[1]
    confidence = note[2]
    remarks = note[3]
    
    idx = cs_meta.index[(cs_meta['Country'] == country) & (cs_meta['Station_id'] == station)][0] # There can be only one
    
    cs_meta.iat[idx,c_note] = remarks
    cs_meta.iat[idx,c_conf] = confidence

In [16]:
# Save the metadata file
cs_meta.to_csv(cs_meta_path / cs_meta_name, encoding='utf-8', index=False)

In [17]:
cs_meta

,Country,Station_id,Station_name,Station_lat,Station_lon,Station_source,Outlet_lat,Outlet_lon,Outlet_source,Mapped_lat,...,Ref_area_1_km2,Ref_area_2_src,Ref_area_2_km2,Ref_shape,Ref_shape_source,Ref_shape_area_km2,Ref_and_new_shape_fractional_overlap,Manual_delineation_notes,Delineation_confidence,Delineation_notes
0,CAN,01AD002,SAINT JOHN RIVER AT FORT KENT,47.25806,-68.59583,WSC 20222 data set,47.257877,-68.594919,WSC 20222 data set,47.257312,...,14700.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,14677.400000,0.993792,NaN,high,n/a
1,CAN,01AD003,ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE,47.20661,-68.95694,WSC 20222 data set,47.206016,-68.956727,WSC 20222 data set,47.205895,...,1350.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,1348.300000,0.985513,NaN,high,n/a
2,CAN,01AE001,FISH RIVER NEAR FORT KENT,47.23750,-68.58278,WSC 20222 data set,47.237438,-68.581991,WSC 20222 data set,47.237438,...,2260.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,2238.950000,0.979374,NaN,high,n/a
3,CAN,01AF007,GRANDE RIVIERE AT VIOLETTE BRIDGE,47.24697,-67.92128,WSC 20222 data set,47.246908,-67.921271,WSC 20222 data set,47.246667,...,339.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,325.196000,0.977787,NaN,high,n/a
4,CAN,01AF009,IROQUOIS RIVER AT MOULIN MORNEAULT,47.45783,-68.35683,WSC 20222 data set,47.457622,-68.356710,WSC 20222 data set,47.457622,...,182.00,HYDAT effective drainage area,NaN,yes,WSC 2022 data set,183.430000,0.986569,NaN,high,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,USA,14309500,"WEST FORK COW CREEK NEAR GLENDALE, OR",42.80400,-123.61091,CAMELS-US,-999.000000,-999.000000,NaN,42.804462,...,224.92,Geospatial fabric area,226.31,yes,CAMELS-US data set (HCDN),226.314346,0.988873,NaN,high,n/a
1694,USA,14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",43.34984,-122.72894,CAMELS-US,-999.000000,-999.000000,NaN,43.349840,...,587.90,Geospatial fabric area,588.01,yes,CAMELS-US data set (HCDN),588.025033,0.995104,NaN,high,n/a
1695,USA,14325000,"SOUTH FORK COQUILLE RIVER AT POWERS, OR",42.89150,-124.07065,CAMELS-US,-999.000000,-999.000000,NaN,42.892325,...,443.07,Geospatial fabric area,444.92,yes,CAMELS-US data set (HCDN),444.925651,0.991051,NaN,high,n/a
1696,USA,14362250,"STAR GULCH NEAR RUCH, OR.",42.15401,-123.07532,CAMELS-US,-999.000000,-999.000000,NaN,42.155000,...,41.42,Geospatial fabric area,43.88,yes,CAMELS-US data set (HCDN),43.877901,0.937207,NaN,high,n/a


## Check if we have notes where we need them
We decided to investigate at least every shape where overlap between the new delineation and the reference shapefile is `< 0.90`.

In [18]:
for ix,row in cs_meta.iterrows():
        
    # Skip to next if we don't have a reference shape
    # This simplifies the next if statement because rows with no reference shape have a 'n/a' value rather than a float 
    #   in the row.Ref_and_new_shape_fractional_overlap field
    if row.Ref_shape == 'no':
        continue 
        
    # Do the actual check
    if (row.Ref_and_new_shape_fractional_overlap < 0.9) and (row.Delineation_notes == 'n/a'):
        print(f'Fractional overlap for basin {row.Station_id} (index {ix}) is {row.Ref_and_new_shape_fractional_overlap} but notes are {row.Delineation_notes}')